In [62]:
import pandas as pd
from google.cloud import storage
from google.oauth2 import service_account
import io
import importlib

import utils as ut

In [ ]:
importlib.reload(ut)

In [63]:
# Descargar los datos del blob y cargarlos en un DataFrame
bucket = ut.set_config

original_blob_path = "webapp/business_reduced.csv"
original_blob = bucket.blob(original_blob_path)
data = original_blob.download_as_bytes()

# Crear el DataFrame
df_bus = pd.read_csv(io.BytesIO(data))

In [64]:
# Imprimir información sobre el DataFrame después de aplicar los filtros
print(df_bus.info())

# Imprimir las primeras filas del DataFrame después de aplicar los filtros
print(df_bus.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   business_id     2000 non-null   object 
 1   name            2000 non-null   object 
 2   address         1925 non-null   object 
 3   city            2000 non-null   object 
 4   state           2000 non-null   object 
 5   postal_code     2000 non-null   object 
 6   latitude        2000 non-null   float64
 7   longitude       2000 non-null   float64
 8   stars           2000 non-null   float64
 9   review_count    2000 non-null   int64  
 10  is_open         2000 non-null   int64  
 11  attributes      1792 non-null   object 
 12  categories      2000 non-null   object 
 13  hours           1687 non-null   object 
 14  business_id.1   0 non-null      float64
 15  name.1          0 non-null      float64
 16  address.1       0 non-null      float64
 17  city.1          0 non-null      f

In [65]:
#Filtrar el DataFrame para obtener solo las columnas que necesitamos
df_bus = df_bus.iloc[:, :14]

In [66]:
#Filtrar los establecimientos por estados específicos
df_bus = df_bus[(df_bus['state'] == 'FL') | (df_bus['state'] == 'CA') | (df_bus['state'] == 'NV')]

In [67]:
#Guardar los distintos establecimientos de los estados ya filtrados
business = df_bus['business_id']

In [68]:
#Exportar / Guardar el DataFrame filtrado ya definito / procesado
processed_blob_path = "processed/business/business_filtered_by_state.csv"
processed_blob = bucket.blob(processed_blob_path)
csv_data = df_bus.to_csv(index=False).encode('utf-8')

processed_blob.upload_from_string(csv_data, content_type='text/csv')

print(f"El DF se guardó correctamente en {processed_blob_path}")

El DF se guardó correctamente en processed/business/business_filtered_by_state.csv


In [69]:
#Eliminar el DataFrame para ahorrar recursos en la memoria
del df_bus